In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from string import punctuation
from nltk.stem.porter import *
from nltk import word_tokenize
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer #word stemmer class
import os
import gensim
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import svm
import xgboost as xgb
XGBClassifier = xgb.XGBClassifier


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import precision_score

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import warnings
warnings.filterwarnings("ignore")
stop_words = list(punctuation)+['``', "'s", "..."]+['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 'can', 'will', 'just', 'don', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',  'ma', 'mightn', 'needn', 'shouldn', "shouldn't"]

In [46]:
#1: unreliable
#0: reliable
df_train=pd.read_csv('train.csv')
df_validation=pd.read_csv('test.csv')

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [47]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:

df_train.isna().sum()    #get na

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [49]:
df_train=df_train.fillna(' ')

In [50]:
df_train['total']=df_train['title']+' '+df_train['author']+df_train['text']
df_validation['total']=df_validation['title']+' '+df_validation['author']+df_validation['text']

In [51]:
from bs4 import BeautifulSoup
from unidecode import unidecode

In [52]:
stopwords


<WordListCorpusReader in '/home/zaonx/nltk_data/corpora/stopwords'>

In [53]:
# function to remove user handles
def remove_pattern (input_txt , pattern):
    r = re.findall(pattern, str(input_txt))
    # remove user handles
    for match in r:
        input_txt= re.sub(match, '', input_txt)
    return input_txt

def remove_non_ascii1(text):
    input_txt = ''.join([i if ord(i) < 128 else ' ' for i in text])
    input_txt = re.sub(r'[^\x00-\x7F]+','', input_txt)
    return input_txt


def handleSmiley(tweet):
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    return tweet

_regex =["_", "@[\w]*", "#","http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))","RT[\s]+" ]

def cleanData(data):
    for reg in _regex:
        data= data.apply(lambda row:remove_pattern(row, reg))
        
#     data = data.apply(lambda row : BeautifulSoup(row, "lxml").text ) #remove html enuty
    data = data.apply(lambda row:remove_non_ascii1(str(row)))
    return data

df_train['tokenized_text'] = cleanData(df_train['total'])

In [54]:
import nltk
nltk.download('punkt')
def tokenizeData(_data):
    _data['tokenized_text'] = _data['tokenized_text'].str.lower() 
    _data['tokenized_text']= _data['tokenized_text'].apply(lambda x: x.strip())
    _data['tokenized_text']= _data['tokenized_text'].apply(lambda row:word_tokenize(row))
    _data['tokenized_text'] = _data['tokenized_text'].apply(lambda row:[word for word in row if len(word) > 2])
    _data['tokenized_text'] = _data['tokenized_text'].apply(lambda row:[word for word in row if word not in stop_words])
    # lemmatizing words
    lamme = WordNetLemmatizer()
    # stemming words
    stemmer = PorterStemmer()
    _data['tokenized_text']= _data['tokenized_text'].apply(lambda x: [stemmer.stem(i) for i in x])
    _data['tokenized_text']= _data['tokenized_text'].apply(lambda x: [lamme.lemmatize(i) for i in x])

    _data['tokenized_text'] = _data['tokenized_text'].apply(lambda x: ' '.join(x))
    _data['tokenized_text'] = _data['tokenized_text'].apply(lambda x: unidecode(x))

    return _data
df_train = tokenizeData(df_train)
df_validation['tokenized_text'] = cleanData(df_validation['total'])
df_validation = tokenizeData(df_validation)

[nltk_data] Downloading package punkt to /home/zaonx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
# import libraries
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split as tts, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [56]:
# tdf = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
# vectorizer = tdf.fit(df_train['tokenized_text'])
# transformed_text = vectorizer.transform(df_train['tokenized_text'])
# transformed_title = vectorizer.transform(df_train['title'].values.astype('U'))

In [57]:
# def get_tfidf_term_scores(feature_names):
#     '''Returns dictionary with term names and total tfidf scores for all terms in corpus'''
#     term_corpus_dict = {}
#     # iterate through term index and term 
#     for term_ind, term in enumerate(feature_names):
#         term_name = feature_names[term_ind]
#         term_corpus_dict[term_name] = np.sum(transformed_title.T[term_ind].toarray())
        
#     return term_corpus_dict

In [58]:
# #list of features created by tfidf
# feature_names = tdf.get_feature_names()

In [59]:
# term_corpus_dict = get_tfidf_term_scores(feature_names)


In [60]:
# def get_sorted_tfidf_scores(term_corpus_dict):
#     '''Returns sort words from highest score to lowest score'''
#     # sort indices from words wit highest score to lowest score
#     sortedIndices = np.argsort( list(term_corpus_dict.values()))[::-1]
#     # move words and score out of dicts and into arrays
#     termNames = np.array(list(term_corpus_dict.keys()))
#     scores = np.array(list(term_corpus_dict.values()))
#     # sort words and scores
#     termNames = termNames[sortedIndices]
#     scores = scores[sortedIndices]
    
#     return termNames, scores



In [61]:
# termNames, scores = get_sorted_tfidf_scores(term_corpus_dict)

In [62]:
# def plot_tfidf_scores(scores,termNames, n_words = 18):
#     '''Returns one plot for Importance of Top N Terms
#        and one plot for Importance of Select K Terms'''

#     # Create a figure instance, and the two subplots
#     fig = plt.figure(figsize = (14, 18))
    
#     override = {'fontsize': 'large'}

#     fig.add_subplot(221)   
#     n_words = 75
#     sb.set()
#     sb.barplot(x = scores[:n_words], y = termNames[:n_words]);
#     plt.title(" Top tfidf score words".format(n_words));
#     plt.xlabel("TFIDF Score");

In [63]:
# plot_tfidf_scores(scores, termNames, n_words = 18)

In [64]:
X_train, X_test, y_train, y_test = tts(df_train['tokenized_text'],df_train['label'],test_size = 0.2,random_state = 0)

In [65]:
_decisionTree = DecisionTreeClassifier()
_gaussian = GaussianNB()
_svc=svm.LinearSVC(C=1.0)
_randomForest = RandomForestClassifier(bootstrap=True, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, 
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
_decisionTree = DecisionTreeClassifier(class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, 
                       random_state=None, splitter='best')
_baggingClassifier = BaggingClassifier(base_estimator=DecisionTreeClassifier(
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        random_state=None,
                                                        splitter='best'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=10, n_jobs=None,
                  oob_score=False, random_state=0, verbose=0, warm_start=False)
_logisticRegression = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=150,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
_xgBoost = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8045997961875188,
              gamma=0.04808827554571038, learning_rate=0.31215697934688114,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
              n_estimators=138, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=0.9746919954946938, verbosity=1)
_gradientBoost = GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=8,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=50,
                                                  min_samples_split=500,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  random_state=10,
                                                  subsample=0.8, tol=0.0001,
                                                  validation_fraction=0.1,
                                                  verbose=0, warm_start=False)

In [66]:
def setModels(_model, _exportTo, _name):
    print("*"*10+_name+"*"*10)
    dt_pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', _model)])
    dt_pipeline.fit(X_train, y_train)
    dt_predicted = dt_pipeline.predict(X_test)
    print(accuracy_score(dt_predicted,y_test))
    print(classification_report(y_test, dt_predicted))
    test_predictions = dt_pipeline.predict(df_validation['tokenized_text'])
    output = pd.DataFrame({'id': df_validation['id'],'label': test_predictions})
    output.to_csv(_exportTo, index=False)
    

In [67]:
setModels(_decisionTree, 'submissions/submission_dtc.csv', "decisionTree")
#setModels(_gaussian, 'submissions/submission_gaussian.csv', "gaussian")
setModels(_svc, 'submissions/submission_svc.csv', "svc")
setModels(_randomForest, 'submissions/submission_ranfor.csv', "randomForest")
setModels(_baggingClassifier, 'submissions/submission_baggClass.csv', "baggingClassifier")
setModels(_logisticRegression, 'submissions/submission_logreg.csv', "logisticRegression")
setModels(_xgBoost, 'submissions/submission_xgboost.csv', "xgBoost")
setModels(_gradientBoost, 'submissions/submission_gradientboost.csv', "gradientBoost")


**********decisionTree**********
0.9502403846153846
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      2046
           1       0.95      0.95      0.95      2114

    accuracy                           0.95      4160
   macro avg       0.95      0.95      0.95      4160
weighted avg       0.95      0.95      0.95      4160

**********svc**********
0.9735576923076923
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2046
           1       0.97      0.98      0.97      2114

    accuracy                           0.97      4160
   macro avg       0.97      0.97      0.97      4160
weighted avg       0.97      0.97      0.97      4160

**********randomForest**********
0.9384615384615385
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      2046
           1       0.96      0.91      0.94      2114

    accuracy                       